In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import pymongo

import pandas as pd
import numpy as np

from pubcrawler.article import Article
from epitator.annodoc import AnnoDoc
from epitator.geoname_annotator import GeonameAnnotator

from reporter import Reporter

client = pymongo.MongoClient("localhost", 27017)
articles = client.pmc.articles

geoname_annotator = GeonameAnnotator()

Find one article and count geonames

In [19]:
n = 25

cursor = articles.find({}, limit = n)
reporter = Reporter(5, n)

rows = []
for idx, record in enumerate(cursor):
    row = {}
    article = AnnoDoc(record["extracted_text"]).add_tier(geoname_annotator)
    geospans = article.tiers["geonames"]
    row["text_matches"] = record.get("text_matches")
    row["n_spans"] = len(geospans)
    row["n_spans_over90"] = sum([1 for span in geospans if span.metadata["geoname"].score > 0.9])
    rows.append(row)
    reporter.report(idx)

df = pd.DataFrame.from_records(rows)
df["any_matches"] = [False if row is None else True for row in df["text_matches"]]

Processed 25 articles (100.0%) in 1m27s; about 0m0s left.


In [20]:
df

,n_spans,n_spans_over90,text_matches,any_matches
0,26,19,[study site],True
1,2,1,None,False
2,0,0,None,False
3,4,0,None,False
4,10,6,None,False
5,0,0,None,False
6,10,4,None,False
7,30,19,None,False
8,1,0,None,False
9,5,2,None,False


Sample specifically articles matching terms

In [71]:
with open("terms") as f:
    terms = [line.strip() for line in f.readlines()]

In [29]:
n = 250

In [32]:
terms_cursor = articles.find({ "text_matches": { "$in": terms } }, limit = n)
noterms_cursor = articles.find({ "text_matches": { "$not": {"$in": terms } } }, limit = n)
sample = list(terms_cursor) + list(noterms_cursor)

In [33]:
reporter = Reporter(5, len(sample))

rows = []
for idx, record in enumerate(sample):
    row = {}
    article = AnnoDoc(record["extracted_text"]).add_tier(geoname_annotator)
    geospans = article.tiers["geonames"]
    row["text_matches"] = record.get("text_matches")
    row["n_spans"] = len(geospans)
    row["n_spans_over90"] = sum([1 for span in geospans if span.metadata["geoname"].score > 0.9])
    rows.append(row)
    reporter.report(idx)

df = pd.DataFrame.from_records(rows)
df["any_matches"] = [False if row is None else True for row in df["text_matches"]]

Processed 500 articles (100.0%) in 31m57s; about 0m0s left.


In [56]:
df.to_csv(os.path.join("data", "geonames.csv"))

In [57]:
df.loc[df["any_matches"]].describe()

           n_spans  n_spans_over90
count   250.000000      250.000000
mean     52.204000       19.236000
std     168.625826       49.785608
min       0.000000        0.000000
25%      11.000000        3.000000
50%      24.000000        8.000000
75%      45.750000       20.000000
max    2448.000000      680.000000


In [58]:
df.loc[~df["any_matches"]].describe()

          n_spans  n_spans_over90
count  250.000000      250.000000
mean    12.508000        5.088000
std     17.606188        9.266267
min      0.000000        0.000000
25%      2.000000        0.000000
50%      7.000000        1.000000
75%     15.000000        5.000000
max    107.000000       59.000000
